In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

drev = pd.read_csv("../../web/exports/drev.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Identifiant': 'str', 'Campagne': 'str', 'Siret Opérateur': 'str', 'Code postal Opérateur': 'str'}, low_memory=False)
etablissements = pd.read_csv("../../web/exports/etablissements.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Login': 'str', 'Identifiant etablissement': 'str'}, index_col=False, low_memory=False)
dr = pd.read_csv("../../web/exports/dr.csv", encoding="iso8859_15", delimiter=";",decimal=",", dtype={'Identifiant': 'str', 'Campagne': 'str', 'Valeur': 'float64'}, low_memory=False)


In [ ]:
campagne = '2020'
drev = drev.query("Campagne == @campagne");

dr = dr.query("Campagne == @campagne");
dr = dr.query("Appellation != 'CDR' and Appellation != 'CVG'")

dr["Bailleur PPM"] = dr['Bailleur PPM'].fillna("DEFAUT")

dr = pd.pivot_table(dr, values= 'Valeur', index=['Identifiant',"Bailleur PPM","Appellation","INAO","Produit"],columns=['Categorie'], aggfunc=np.sum)

drev_cru = drev.query("Appellation != 'CDR' and Appellation != 'CVG'")

drev_cru_with_dr = pd.merge(drev_cru, dr, how='inner',left_on="Identifiant", right_on="Identifiant",suffixes=("", " societe"))

drev_cru_with_dr['AOCIDBAIL'] = drev_cru_with_dr['Appellation']+'-'+drev_cru_with_dr['Identifiant']
#print(drev_cru_with_dr)
#remettre les colonnes dans l'ordre comme présenter par Serge et regarder pk le merge ne fonctionne pas entre dr et drev_cru, une ligne en trop par exemple n°328

In [ ]:
drev_cru_with_dr.to_csv('../../web/exports/test_cru'+campagne+'.csv', encoding="iso8859_15", sep=";", decimal=",")